# Assignment 2

## SQL and NoSQL

### Exercise 1

_Task: Establish connection to MongoDB and SQLite_

#### Connection to SQLite

In [18]:
import sqlite3 as lite

In [19]:

def get_sqlite_connection():
    conn = None
    conn = lite.connect('northwind.db')
    conn.text_factory = str
    return conn

In [22]:
conn = get_sqlite_connection()
cur = conn.cursor()
cur.execute('SELECT * FROM PRODUCTS')
cur.fetchall()[:1]

[(1, 'Chai', 1, 1, '10 boxes x 20 bags', 18, 39, 0, 10, '0')]

In [35]:
cur.execute('SELECT * FROM customers ORDER BY customerid ASC')
cur.fetchall()[:1]

[('ALFKI',
  'Alfreds Futterkiste',
  'Maria Anders',
  'Sales Representative',
  'Obere Str. 57',
  'Berlin',
  None,
  '12209',
  'Germany',
  '030-0074321',
  '030-0076545')]

#### Connection to MongoDB

In [24]:
from pymongo import MongoClient
import pymongo

In [8]:
def get_mongo_connection():
    client=MongoClient('localhost',27017)
    return client['Northwind']   # Get the database

In [9]:
c = get_mongo_connection()

In [13]:
products = c.products.find()
list(products)[:1]

[{u'CategoryID': 1,
  u'Discontinued': 0,
  u'ProductID': 1,
  u'ProductName': u'Chai',
  u'QuantityPerUnit': u'10 boxes x 20 bags',
  u'ReorderLevel': 10,
  u'SupplierID': 1,
  u'UnitPrice': 18.0,
  u'UnitsInStock': 39,
  u'UnitsOnOrder': 0,
  u'_id': ObjectId('5609b12899ded9537b06c7e5')}]

In [60]:
c = get_mongo_connection()
customers = c.customers.find().sort('CustomerID', pymongo.ASCENDING)
top10 = [customer['CustomerID'] for customer in customers][:10]
pd.DataFrame(top10, columns=['CustomerID'])

,CustomerID
0,ALFKI
1,ANATR
2,ANTON
3,AROUT
4,BERGS
5,BLAUS
6,BLONP
7,BOLID
8,BONAP
9,BOTTM


### Exercise 2

In [62]:
cur.execute("SELECT * FROM customers AS c INNER JOIN orders AS o ON c.customerid=o.customerid where o.customerid='ALFKI'")
pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1997-09-22 00:00:00.000,1997-09-02 00:00:00.000,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
1,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1997-10-31 00:00:00.000,1997-10-13 00:00:00.000,2,61.02,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
2,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1997-11-24 00:00:00.000,1997-10-21 00:00:00.000,1,23.94,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
3,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1998-02-12 00:00:00.000,1998-01-21 00:00:00.000,3,69.53,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
4,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1998-04-27 00:00:00.000,1998-03-24 00:00:00.000,1,40.42,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
5,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,1998-05-07 00:00:00.000,1998-04-13 00:00:00.000,1,1.21,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany


In [53]:
for customer in c.customers.find({'CustomerID':'ALFKI'}):
    for order in c.orders.find({'CustomerID': customer['CustomerID']}):
        for details in c['order-details'].find({'OrderID': order['OrderID']}):
            joined_result = order.copy()
            joined_result.update(details)
r = cur.fetchall()

# Pretty print using pandas
pd.DataFrame([joined_result.values()], columns=joined_result.keys())

,OrderID,ShipVia,ShippedDate,ShipName,EmployeeID,ShipPostalCode,ShipCity,ShipRegion,OrderDate,CustomerID,Discount,Quantity,Freight,RequiredDate,_id,UnitPrice,ShipAddress,ProductID,ShipCountry
0,11011,1,1998-04-13 00:00:00.000,Alfred's Futterkiste,3,12209,Berlin,NULL,1998-04-09 00:00:00.000,ALFKI,0,20,1.21,1998-05-07 00:00:00.000,5609b12899ded9537b06c3f0,21.5,Obere Str. 57,71,Germany


### Exercise 3
_Get all orders (with products) made by ALFKI that contain at least 2 products._

In [68]:
import pandas as pd

In [80]:
cur.execute('SELECT * FROM customers AS c INNER JOIN orders AS o ON c.customerid=o.customerid '
                    ' INNER JOIN "Order Details" AS od ON od.orderid=o.orderid '
                    ' INNER JOIN products AS p ON p.productid=od.productid '
                    " WHERE c.customerid='ALFKI'"
                    ' GROUP BY o.orderid HAVING count(distinct od.productid) >= 2')
r = cur.fetchall()
pd.DataFrame(r)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,46,Spegesild,21,8,4 - 450 g glasses,12.0,95,0,0,0
1,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,76,Lakkalik��ri,23,1,500 ml,18.0,57,0,20,0
2,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,77,Original Frankfurter gr�ne So�e,12,2,12 boxes,13.0,32,0,15,0
3,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,28,R�ssle Sauerkraut,12,7,25 - 825 g cans,45.6,26,0,0,1
4,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,71,Flotemysost,15,4,10 - 500 g pkgs.,21.5,26,0,0,0


In [85]:
from bson.code import Code
c = get_mongo_connection()
reducer = Code(
        """
            function(obj, prev){
              prev.count++;
            }
        """)
orders = []
for customer in c.customers.find({'CustomerID':'ALFKI'}):
    for order in c.orders.find({'CustomerID': customer['CustomerID']}):
        results = c['order-details'].group(key={"OrderID": 1}, condition={}, initial={"count": 0}, reduce=reducer)
        for details in results:
            if details['OrderID'] == order['OrderID'] and details['count'] >= 2:
                orders.append(order)
                joined_result = order.copy()
                joined_result.update(details)
                
# Pretty print using pandas
pd.DataFrame(orders)

,CustomerID,EmployeeID,Freight,OrderDate,OrderID,RequiredDate,ShipAddress,ShipCity,ShipCountry,ShipName,ShipPostalCode,ShipRegion,ShipVia,ShippedDate,_id
0,ALFKI,6,29.46,1997-08-25 00:00:00.000,10643,1997-09-22 00:00:00.000,Obere Str. 57,Berlin,Germany,Alfreds Futterkiste,12209,NULL,1,1997-09-02 00:00:00.000,5609b12899ded9537b06c632
1,ALFKI,4,23.94,1997-10-13 00:00:00.000,10702,1997-11-24 00:00:00.000,Obere Str. 57,Berlin,Germany,Alfred's Futterkiste,12209,NULL,1,1997-10-21 00:00:00.000,5609b12899ded9537b06c66d
2,ALFKI,1,69.53,1998-01-15 00:00:00.000,10835,1998-02-12 00:00:00.000,Obere Str. 57,Berlin,Germany,Alfred's Futterkiste,12209,NULL,3,1998-01-21 00:00:00.000,5609b12899ded9537b06c6f2
3,ALFKI,1,40.42,1998-03-16 00:00:00.000,10952,1998-04-27 00:00:00.000,Obere Str. 57,Berlin,Germany,Alfred's Futterkiste,12209,NULL,1,1998-03-24 00:00:00.000,5609b12899ded9537b06c767
4,ALFKI,3,1.21,1998-04-09 00:00:00.000,11011,1998-05-07 00:00:00.000,Obere Str. 57,Berlin,Germany,Alfred's Futterkiste,12209,NULL,1,1998-04-13 00:00:00.000,5609b12899ded9537b06c7a2


#### Exercise 4

In [ ]:
class TestExercise54(unittest.TestCase):
    def test_sqlite_count_products(self):
        conn = connect_sqlite()
        cur = conn.cursor()
        cur.execute('SELECT count(productid) FROM "Order Details" where productid=7')
        print 'SQLite Count: ' + str(cur.fetchone()[0])

    def test_mongo_count_products(self):
        c = get_mongo_connection()
        print 'MongoDB Count: ' + str(c['order-details'].find({'ProductID': 7}).count())

#### Exercise 5

In [ ]:
class TestExercise55(unittest.TestCase):


    def test_sqlite_distinct(self):
        conn = connect_sqlite()
        cur = conn.cursor()
        stmt = """
        SELECT count(p.productid), p.productname FROM orders
        INNER JOIN [Order Details] AS od ON od.orderid=orders.orderid
        INNER JOIN products AS p ON p.productid=od.productid
        WHERE CustomerID IN
                    (SELECT CustomerID FROM orders
					 INNER JOIN [Order Details] ON [Order Details].orderid=orders.orderid
					 WHERE [Order Details].productid=7)
		GROUP BY od.productid
        """
        cur.execute(stmt)
        for r in cur.fetchall():
            print r

    def test_mongo_distinct(self):
        c = get_mongo_connection()
        from bson.code import Code
        reducer = Code(
                """
                    function(obj, prev){
                      prev.count++;
                    }
                """)
        #results = c['order-details'].group(key={"ProductID": 1, 'CustomerID':2}, condition={}, initial={"count": 0}, reduce=reducer)
        #for x in results:
        #    print x
        customers = {}
        products = {}
        for details in c['order-details'].find({'ProductID': 7}):
            for order in c.orders.find({'OrderID':details['OrderID']}):
                customers[order['CustomerID']]= 0

        for customer_id in customers.keys():
            for r in c.orders.find():
                if r['CustomerID'] == customer_id:
                    for details in c['order-details'].find({'OrderID':r['OrderID']}):

                        product_name = c.products.find({'ProductID': details['ProductID']})[0]['ProductName']

                        if product_name in products.keys():
                            products[product_name] += 1
                        else:
                            products[product_name] = 1

        print products